# MLG HW2

In [1]:
import pandas as pd
import numpy as np
import os, time, torch, json
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# from torch_geometric.utils import accuracy,sparse_mx_to_torch_sparse_tensor
import torch_geometric.utils 
# from models.GCN import GCN
import scipy.sparse as sp
from tqdm import tqdm, trange
from torch.autograd import Variable
import torchvision
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
import torch.utils.data as Data
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from scipy.stats import entropy
from sklearn.preprocessing import MinMaxScaler

# test 
from sklearn.metrics import roc_auc_score, average_precision_score


/home/rita/miniconda3/envs/jupyterlab/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
content = []
test = []
train = []
upload = []
edge_index = [] 
for i in range(3):
    os.chdir('/home/rita/111/111-2MLG/HW2/dataset{}'.format(i + 1))
    temp = pd.read_csv('./content.csv', header = None, sep = '\t')
    temp.sort_values(by = [0], inplace = True)
    temp.set_index([0], inplace = True)
    temp = torch.Tensor(np.array(temp)).to(torch.float32)
    print(temp.shape)
    content.append(temp)
    test.append(pd.read_csv('./test.csv', header = None))
    temp = pd.read_csv('./train.csv')
    print(temp.shape)
    train.append(temp)
    temp = temp[temp.label == 1]
    temp = temp[['to', 'from']]
    temp = temp.reset_index(drop = True)
    print(temp.shape)
    edge_index.append(temp)
    upload.append(pd.read_csv('./upload.csv'))
print('content :\n', content[0][:2])
print('test :\n', test[0].head(2))
print('train : \n', train[0].head(2))
print('edge_index : \n', edge_index[0].head(2))
print('upload : \n', upload[0].head(2))
os.chdir('/home/rita/111/111-2MLG/HW2')

torch.Size([2708, 1433])
(8686, 4)
(4324, 2)
torch.Size([3312, 3703])
(7544, 4)
(3736, 2)
torch.Size([877, 1703])
(2572, 4)
(1273, 2)
content :
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
test :
         0     1     2
0      id    to  from
1  E10559  2323  2673
train : 
        id    to  from  label
0  E10311  2399  2339      0
1  E10255  2397  1144      1
edge_index : 
      to  from
0  2397  1144
1  2450  1312
upload : 
        id  prob
0  E10559   0.5
1   E4849   0.5


In [17]:
# content preprocessing
new_features = []
for i in range(len(content)) :
    t = content[i]
    features_entropy = []
    for i in range(t.shape[1]) :
        temp = t.T[i]
        t1 = torch.sum(temp == 0) / len(temp)
        t2 = torch.sum(temp == 1) / len(temp)
        temp = torch.tensor([t1, t2])
        temp = entropy(temp)
        if (temp == 0) :
            temp = 0
        else :
            temp = 1 / temp
        features_entropy.append(temp)
    features_entropy = torch.tensor(features_entropy).reshape(1, -1)
    t = t * features_entropy
    t = t.type(torch.float32)
    new_features.append(t)
print(new_features[0][:2])

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [5]:
device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))
print(f"Training on device {device}.") #可以根據輸出結果知道是否有可用的GPU

Training on device cuda.


In [18]:
def get_ADJ(content, edge_index):
    
    s = content.shape[0]
    m = np.zeros((s, s))
    for i in range(edge_index.shape[0]):
        x = edge_index.iat[i, 0]
        y = edge_index.iat[i, 1]
        m[x, y] = 1
        m[y, x] = 1

    return m




In [ ]:
def load_data1(dataset1):
matrix =np.zeros( (2708, 2708) )
#對應train資料有連接上則給予1，無則0
for index,row in dataset1.iterrows():
if row[3]==1:
x = row[1]
y = row[2]
matrix[x,y] = 1
else:
x = row[1]
y = row[2]
matrix[x,y] = 0
return matrix
matrix1 = load_data1(dataset1)
matrix1 = pd.DataFrame(matrix1)
#因為有from跟to特徵所以在做一個to的adjacency matrix
def load_data2(dataset1):
matrix =np.zeros( (2708, 2708) )
for index,row in dataset1.iterrows():
if row[3]==1:
x = row[1]
y = row[2]
matrix[y,x] = 1
else:
x = row[1]
y = row[2]
matrix[y,x] = 0
return matrix
#再將兩個matrix合併
matrix2 = load_data2(dataset1)
matrix2 = pd.DataFrame(matrix2)
matrix = pd.concat([matrix1,matrix2],axis=1)

In [ ]:
#再將content & adjacency matrix 合併成為最後想測試的資料特徵
contentfinal = pd.concat([contentArray_sort,matrix],axis=1)
contentfinal = contentfinal.values
contentfinal = pd.DataFrame(contentfinal)

In [ ]:
#造to矩陣
def load_data_to(dataset1):
data = []
for index, row in dataset1.iterrows():
x = row[1]
data.append(contentfinal.iloc[x,1:])
return data
df_to = load_data_to(dataset1)
df_to = pd.DataFrame(df_to)
df_to = df_to.values
df_to = pd.DataFrame(df_to)
造from矩陣
def load_data_from(dataset1):
data = []
for index, row in dataset1.iterrows():
x = row[2]
data.append(contentfinal.iloc[x,1:])
return data
df_from = load_data_from(dataset1)
df_from = pd.DataFrame(df_from)
df_from = df_from.values
df_from = pd.DataFrame(df_from)
#將兩矩陣合併成測試資料
df = pd.concat([df_to,df_from],axis=1)
df = df.values
df = pd.DataFrame(df)

In [ ]:
data = df.iloc[:,:13698].values
data = pd.DataFrame(data)
label = dataset1.loc[:,'label'].values
label = pd.DataFrame(label)

In [ ]:
from keras.models import Model
from keras.layers import Dense, Input
encoding_dim = 100
# this is our input placeholder
input_img = Input(shape=(13698,))
# encoder layers
encoded = Dense(1024, activation='relu')(input_img)
encoded = Dense(128, activation='relu')(encoded)
encoded = Dense(20, activation='relu')(encoded)
encoder_output = Dense(encoding_dim)(encoded)
# decoder layers
decoded = Dense(20, activation='relu')(encoder_output)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(1024, activation='relu')(decoded)
decoded = Dense(13698, activation='tanh')(decoded)
# construct the autoencoder model
autoencoder = Model(input=input_img, output=decoded)
# construct the encoder model for plotting
encoder = Model(input=input_img, output=encoder_output)
# compile autoencoder
autoencoder.compile(optimizer='adam', loss='mse')
# training
autoencoder.fit(x_train, x_train,
                epochs=20,
                batch_size=256,
                shuffle=True)
# plotting
encoded_imgs = encoder.predict(x_test)
plt.scatter(encoded_imgs[:, 0], encoded_imgs[:, 1], c=y_test)
plt.colorbar()
plt.show()

In [ ]:
classifier = Sequential()
#First Hidden Layer
classifier.add(Dense(512, activation='relu', kernel_initializer='random_normal', input_dim=13698))
#Second  Hidden Layer
classifier.add(Dense(256, activation='relu', kernel_initializer='random_normal'))
#Third  Hidden Layer
classifier.add(Dense(64, activation='relu', kernel_initializer='random_normal'))
#Forth  Hidden Layer
classifier.add(Dense(20, activation='relu', kernel_initializer='random_normal'))
#Output Layer
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))
#Compiling the neural network
classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])
classifier.fit(auto_x_train,label, batch_size=256, epochs=100)

In [ ]:
#test資料前處裡
test1 = pd.read_csv("C:\\Users\\eagleuser\\Desktop\\hw1_data_update2\\dataset1\\test.csv")
test1.id = test1.id.str.replace('E',"")
test1
dd_to = load_data_to(test1)
dd_to = pd.DataFrame(dd_to)
dd_to = dd_to.values
dd_to = pd.DataFrame(dd_to)
dd_from = load_data_from(test1)
dd_from = pd.DataFrame(dd_from)
dd_from = dd_from.values
dd_from = pd.DataFrame(dd_from)
dd = pd.concat([dd_to,dd_from],axis=1)
dd = dd.values
dd = pd.DataFrame(dd)
dd_test = pd.concat([dd,test1],axis=1)
#放入autoencoder
prob = encoder.predict(dd)
auto_dd = autoencoder.predict(dd)
auto_dd = pd.DataFrame(auto_dd)
y_pred=classifier.predict(auto_dd)
y_pred =(y_pred>0.5)
y_pred
data_test_label = classifier.predict_proba(auto_dd)
data_test_label = pd.DataFrame(data_test_label)
test1 = pd.read_csv("C:\\Users\\eagleuser\\Desktop\\hw1_data_update2\\dataset1\\test.csv")
result=pd.concat([test1,data_test_label],axis=1,join_axes=[test1.index])
#存取測試資料
result.to_csv('C:\\Users\\eagleuser\\Desktop\\hw1_data_update2\\resulut111.csv')

In [ ]:
AUC_1 0.7592

AP_1 0.7596

AUC_2 0.7404

AP_2 0.7720

AUC_3 0.8866

AP_3 0.9011